# ***gemma-medical-qa-lora***
# ***Aziz Deniz Akmermer 220212037***
# ***Yapay Zeka Mühendisliği 4.sınıf***

# **Modeli hemen denemek için :** **Plug-and-Play Model Loading (No Training** **Required)**

# **Environment Setup & Dependencies**


In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U datasets
%pip -q install scikit-learn
%pip -q install wordcloud

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Imports & Configuration**

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging
)
from peft import LoraConfig, get_peft_model
from huggingface_hub import login
from trl import SFTTrainer
import bitsandbytes as bnb
import wandb
from google.colab import drive
import numpy as np
from wordcloud import WordCloud
import re
import pandas as pd
import plotly.express as px
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from IPython.display import display, HTML
from datasets import load_from_disk

## **Dataset Loading & Google Drive Setu**p

In [ ]:
from google.colab import drive
from datasets import load_dataset, load_from_disk
import os

drive.mount("/content/drive")

# Proje yolu (sende sabit)
PROJE_DIZINI = "/content/drive/MyDrive/Colab Notebooks/MR/gemmaqa_project"
VERISETI_DIZINI = os.path.join(PROJE_DIZINI, "data", "dataset_saved")

# Dataset Drive'da var mı?
if os.path.exists(VERISETI_DIZINI):
    ds = load_from_disk(VERISETI_DIZINI)
    kaynak = "Drive (load_from_disk)"
else:
    ds = load_dataset("lavita/ChatDoctor-HealthCareMagic-100k")
    os.makedirs(os.path.dirname(VERISETI_DIZINI), exist_ok=True)
    ds.save_to_disk(VERISETI_DIZINI)
    kaynak = "Hugging Face (ilk indirme + Drive'a kaydedildi)"

print("Dataset kaynağı:", kaynak)
print("Dataset yolu:", VERISETI_DIZINI)
print("Split'ler:", list(ds.keys()))
for split in ds:
    print(split, "örnek sayısı:", len(ds[split]))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset kaynağı: Drive (load_from_disk)
Dataset yolu: /content/drive/MyDrive/Colab Notebooks/MR/gemmaqa_project/data/dataset_saved
Split'ler: ['train']
train örnek sayısı: 112165


# **Data Analysis**

## **Dataset Summary and Examples**

In [ ]:
def kutu(baslik, icerik_html):
    return HTML(f"""
    <div style="
        border:1px solid #333;
        border-radius:12px;
        padding:14px 16px;
        margin:10px 0;
        background:#111;
        color:#eaeaea;
        font-family: ui-sans-serif, system-ui, -apple-system;
    ">
      <div style="font-size:16px; font-weight:600; margin-bottom:8px;">{baslik}</div>
      <div style="font-size:13px; line-height:1.4;">{icerik_html}</div>
    </div>
    """)

train = ds["train"]
kolonlar = train.column_names

display(kutu("Dataset Özeti", f"""
<b>Split:</b> train<br/>
<b>Örnek sayısı:</b> {len(train):,}<br/>
<b>Kolonlar:</b> {", ".join(kolonlar)}
"""))

ornek_sayisi = 5
df_ornek = pd.DataFrame([train[i] for i in range(ornek_sayisi)])
pd.set_option("display.max_colwidth", 300)

display(kutu("İlk Örnekler (ilk 5 satır)", "Aşağıdaki tabloda ham veri gösteriliyor (dataset değişmez)."))
display(df_ornek)

,instruction,input,output
0,"If you are a doctor, please answer the medical questions based on the patient's description.","I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down...","Hi, Thank you for posting your query. The most likely cause for your symptoms is benign paroxysmal positional vertigo (BPPV), a type of peripheral vertigo. In this condition, the most common symptom is dizziness or giddiness, which is made worse with movements. Accompanying nausea and vomiting a..."
1,"If you are a doctor, please answer the medical questions based on the patient's description.",My baby has been pooing 5-6 times a day for a week. In the last few days it has increased to 7 and they are very watery with green stringy bits in them. He does not seem unwell i.e no temperature and still eating. He now has a very bad nappy rash from the pooing ...help!,Hi... Thank you for consulting in Chat Doctor. It seems your kid is having viral diarrhea. Once it starts it will take 5-7 days to completely get better. Unless the kids having low urine output or very dull or excessively sleepy or blood in motion or green bilious vomiting...you need not worry. ...
2,"If you are a doctor, please answer the medical questions based on the patient's description.","Hello, My husband is taking Oxycodone due to a broken leg/surgery. He has been taking this pain medication for one month. We are trying to conceive our second baby. Will this medication afect the fetus? Or the health of the baby? Or can it bring birth defects? Thank you.","Hello, and I hope I can help you today.First, there is no medication that can be taken by the father that has any way to get into your system or a baby if you conceive. Medications can only affect a fetus if you take it while pregnant. The only issue is that certain medications may decrease a m..."
3,"If you are a doctor, please answer the medical questions based on the patient's description.","lump under left nipple and stomach pain (male) Hi,I have recently noticed a few weeks ago a lump under my nipple, it hurts to touch and is about the size of a quarter. Also I have bern experiencing stomach pains that prevent me from eating. I immediatly feel full and have extreme pain. Please help","HI. You have two different problems. The lump under the nipple should be removed, biopsied. This will help you to get rid of the disease, and you get a diagnosis. Second problem looks a bit serious one"
4,"If you are a doctor, please answer the medical questions based on the patient's description.",I have a 5 month old baby who is very congested with a terrible cough. Its rattly/raspy and croupy sounding cough. She started choking on her coughs and the mucous that has come up. She also has a fever and runny nose. Should i take her to urgent care?,Thank you for using Chat Doctor. I would suggest that you see your doctor. Your baby maybe having bronchiolitis which is a lung infection common to your kids age. It is commonly caused by a virus. Albuterol via nebulization should be utilized in order to alleviate the wheezing and also help with...


## **Preparing Q&A Data for Analysis**

In [ ]:
train = ds["train"]
print("Kolonlar:", train.column_names)
def kolon_bul(adaylar, kolonlar):
    for a in adaylar:
        if a in kolonlar:
            return a
    return None

SORU_KOLONU = kolon_bul(["input", "question", "prompt", "instruction", "query"], train.column_names)
CEVAP_KOLONU = kolon_bul(["output", "answer", "response", "completion"], train.column_names)

if SORU_KOLONU is None or CEVAP_KOLONU is None:
    raise ValueError(f"Soru/Cevap kolonları bulunamadı. Mevcut kolonlar: {train.column_names}")

print("Soru kolonu:", SORU_KOLONU)
print("Cevap kolonu:", CEVAP_KOLONU)

df = train.to_pandas()

print("EDA df oluşturuldu:", df.shape)

Kolonlar: ['instruction', 'input', 'output']
Soru kolonu: input
Cevap kolonu: output
EDA df oluşturuldu: (112165, 3)


## **Top Keywords Frequency Analysis**

In [ ]:
eksikler = [x for x in ["df", "SORU_KOLONU", "CEVAP_KOLONU"] if x not in globals()]
if eksikler:
    raise NameError(f"Eksik değişken(ler): {', '.join(eksikler)}.")

def top_kelimeler_plot(metin_serisi: pd.Series, baslik: str, topn: int = 25):
    stop = set(ENGLISH_STOP_WORDS)

    sayac = {}
    for t in metin_serisi.astype(str).tolist():
        toks = re.findall(r"[A-Za-z']{2,}", t.lower())
        for tok in toks:
            if tok in stop:
                continue
            sayac[tok] = sayac.get(tok, 0) + 1

    out = pd.DataFrame(sorted(sayac.items(), key=lambda x: x[1], reverse=True)[:topn],
                       columns=["kelime", "frekans"])

    fig = px.bar(out, x="kelime", y="frekans", title=baslik)
    fig.update_layout(width=520, height=340, margin=dict(l=30, r=15, t=45, b=70))
    fig.update_xaxes(tickangle=45)
    return fig

fig_q = top_kelimeler_plot(df[SORU_KOLONU], "Sorularda en sık kelimeler (Top 25)")
fig_a = top_kelimeler_plot(df[CEVAP_KOLONU], "Cevaplarda en sık kelimeler (Top 25)")

html = f"""
<div style="display:grid; grid-template-columns: 1fr 1fr; gap: 10px;">
  <div>{fig_q.to_html(include_plotlyjs='cdn', full_html=False)}</div>
  <div>{fig_a.to_html(include_plotlyjs=False, full_html=False)}</div>
</div>
"""
display(HTML(html))

In [ ]:
# %pip -q install wordcloud

## **Word Cloud Analysis of Answer Texts**

In [ ]:
eksikler = []
if "df" not in globals():
    eksikler.append("df")
if "CEVAP_KOLONU" not in globals():
    eksikler.append("CEVAP_KOLONU")

if eksikler:
    raise NameError(
        f"Eksik değişken(ler): {', '.join(eksikler)}. "
        "Önce df ve CEVAP_KOLONU oluşmalı."
    )

# Metni birleştir (tüm veri)
metin = " ".join(df[CEVAP_KOLONU].astype(str).tolist())

# WordCloud üret
wc = WordCloud(
    width=1200,
    height=600,
    background_color="black",
    collocations=False
).generate(metin)
img = np.array(wc.to_array())
fig_wc = px.imshow(img, title="WordCloud (Cevap Metinleri - Tüm veri)")
fig_wc.update_xaxes(visible=False)
fig_wc.update_yaxes(visible=False)
fig_wc.show()

# **Modeling**

## **Hugging Face Authentication**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

## **Model Loading with BitsAndBytes**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
BASE_MODEL = "google/gemma-2-2b-it"

if not torch.cuda.is_available():
    raise RuntimeError("CUDA bulunamadı. Runtime > Change runtime type > GPU seç.")

cc_major, cc_minor = torch.cuda.get_device_capability()
print(f"GPU compute capability: {cc_major}.{cc_minor}")
torch_dtype = torch.float16  # cc 7.5 için doğru
attn_implementation = "eager"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation,
    token=True,  # notebook_login ile girilen tokenı kullan
)
tokenizer = AutoTokenizer.from_pretrained(
    BASE_MODEL,
    trust_remote_code=True,
    token=True,
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Model ve tokenizer yüklendi.")

GPU compute capability: 7.5


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Model ve tokenizer yüklendi.


## **Identifying LoRA Target Modules**

In [ ]:
import bitsandbytes as bnb

def lora_hedef_modullerini_bul(model):
    linear_cls = bnb.nn.Linear4bit
    hedefler = set()

    for ad, mod in model.named_modules():
        if isinstance(mod, linear_cls):
            hedefler.add(ad.split(".")[-1])

    hedefler.discard("lm_head")
    return sorted(hedefler)

modules = lora_hedef_modullerini_bul(model)

print("LoRA hedef modülleri:", modules)
print("Hedef modül sayısı:", len(modules))

LoRA hedef modülleri: ['down_proj', 'gate_proj', 'k_proj', 'o_proj', 'q_proj', 'up_proj', 'v_proj']
Hedef modül sayısı: 7


## **Attaching LoRA Adapters**

In [ ]:
from peft import LoraConfig, get_peft_model

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 20,766,720 || all params: 2,635,108,608 || trainable%: 0.7881


## **Data Cleaning and Chat Template Formatting**

In [ ]:
train_ds = ds["train"]
def temizle(metin: str) -> str:
    if metin is None:
        return ""
    metin = str(metin)

    # temizlik
    metin = re.sub(r"\b(?:www\.[^\s]+|http\S+)", "", metin)
    metin = re.sub(r"\b(?:Chat Doctor(?:\.com)?(?:\.in)?|www\.(?:google|yahoo)\S*)", "", metin, flags=re.IGNORECASE)
    metin = re.sub(r"\s+", " ", metin).strip()
    return metin

def satiri_formatla(ornek):
    instruction = temizle(ornek.get("instruction", ""))
    soru = temizle(ornek.get("input", ""))
    cevap = temizle(ornek.get("output", ""))

    if instruction:
        user_icerik = f"{instruction}\n\n{soru}"
    else:
        user_icerik = soru

    mesajlar = [
        {"role": "user", "content": user_icerik},
        {"role": "assistant", "content": cevap},
    ]

    text = tokenizer.apply_chat_template(mesajlar, tokenize=False)
    return {"text": text}

dataset_ft = train_ds.map(
    satiri_formatla,
    remove_columns=train_ds.column_names
)
print(dataset_ft)
print("\nÖrnek text:\n")
print(dataset_ft[0]["text"][:800])

Map:   0%|          | 0/112165 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 112165
})

Örnek text:

<bos><start_of_turn>user
If you are a doctor, please answer the medical questions based on the patient's description.

I woke up this morning feeling the whole room is spinning when i was sitting down. I went to the bathroom walking unsteadily, as i tried to focus i feel nauseous. I try to vomit but it wont come out.. After taking panadol and sleep for few hours, i still feel the same.. By the way, if i lay down or sit down, my head do not spin, only when i want to move around then i feel the whole world is spinning.. And it is normal stomach discomfort at the same time? Earlier after i relieved myself, the spinning lessen so i am not sure whether its connected or coincidences.. Thank you doc!<end_of_turn>
<start_of_turn>model
Hi, Thank you for posting your query. The most likely cause for


## **Training and Evaluation Data Subset**

In [ ]:
TRAIN_N = 5000
EVAL_N  = 500

train_dataset_small = train_dataset.shuffle(seed=42).select(range(TRAIN_N))
eval_dataset_small  = eval_dataset.shuffle(seed=42).select(range(EVAL_N))

print("Train (küçük):", len(train_dataset_small))
print("Eval  (küçük):", len(eval_dataset_small))

Train (küçük): 5000
Eval  (küçük): 500


## **Fine-Tuning Configuration**

In [ ]:
from transformers import TrainingArguments

OUTPUT_DIR = "/content/drive/MyDrive/Colab Notebooks/MR/gemmaqa_project/outputs"

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=500,
    logging_steps=20,
    warmup_steps=10,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    report_to="none",
    load_best_model_at_end=False
)

## **Tokenization and Label Construction**

In [ ]:
from datasets import DatasetDict

small_splits = DatasetDict({
    "train": train_dataset_small,
    "test": eval_dataset_small
})

tokenized_small = small_splits.map(
    tokenize_ve_label,
    batched=True,
    remove_columns=["text"]
)

train_tok = tokenized_small["train"]
eval_tok  = tokenized_small["test"]

print(train_tok)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5000
})


## **Trainer Setup and Data Collation**

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

model.config.use_cache = False

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=eval_tok,
    data_collator=data_collator,
)

print("Trainer hazır.")

Trainer hazır.


## **Model Training and Performance Visualization**

In [ ]:
trainer.train()

model.config.use_cache = True

loglar = trainer.state.log_history
if not loglar:
    raise RuntimeError("log_history boş. logging_steps ayarını ve train() çıktısını kontrol et.")

df_log = pd.DataFrame(loglar)

if "step" not in df_log.columns:
    df_log["step"] = range(1, len(df_log) + 1)

# 1) Training loss
if "loss" in df_log.columns:
    df_loss = df_log.dropna(subset=["loss"]).copy()
    fig_loss = px.line(df_loss, x="step", y="loss", title="Training Loss (step bazında)")
    fig_loss.update_layout(width=900, height=380, margin=dict(l=30, r=15, t=45, b=30))
    fig_loss.show()

# 2) Eval loss (evaluation_strategy steps ise gelir)
if "eval_loss" in df_log.columns:
    df_eval = df_log.dropna(subset=["eval_loss"]).copy()
    fig_eval = px.line(df_eval, x="step", y="eval_loss", title="Eval Loss (step bazında)")
    fig_eval.update_layout(width=900, height=380, margin=dict(l=30, r=15, t=45, b=30))
    fig_eval.show()

# 3) Learning rate
if "learning_rate" in df_log.columns:
    df_lr = df_log.dropna(subset=["learning_rate"]).copy()
    fig_lr = px.line(df_lr, x="step", y="learning_rate", title="Learning Rate (step bazında)")
    fig_lr.update_layout(width=900, height=380, margin=dict(l=30, r=15, t=45, b=30))
    fig_lr.show()

# 4) Hız / performans (varsa)
for kolon, baslik in [("train_runtime", "Train Runtime (s)"),
                      ("train_samples_per_second", "Train Samples/sec"),
                      ("train_steps_per_second", "Train Steps/sec")]:
    if kolon in df_log.columns:
        df_k = df_log.dropna(subset=[kolon]).copy()
        fig_k = px.line(df_k, x="step", y=kolon, title=baslik)
        fig_k.update_layout(width=900, height=380, margin=dict(l=30, r=15, t=45, b=30))
        fig_k.show()

print("Grafikler üretildi. Log satırı sayısı:", len(df_log))


Step,Training Loss,Validation Loss
200,2.266500,2.451904
400,2.367900,2.410964
600,2.232900,2.386013
800,2.201400,2.363704
1000,2.203600,2.343554
1200,2.268100,2.326155
1400,2.118600,2.308455
1600,2.204100,2.293051
1800,2.221600,2.282926
2000,2.230600,2.270660


Grafikler üretildi. Log satırı sayısı: 138


## **Saving LoRA Adapters and Tokenizer**

In [ ]:
MODEL_ADI = "gemma-medical-qa-lora"

trainer.model.save_pretrained(MODEL_ADI)
tokenizer.save_pretrained(MODEL_ADI)
print("LoRA adapter ve tokenizer kaydedildi:", MODEL_ADI)

LoRA adapter ve tokenizer kaydedildi: gemma-medical-qa-lora


# **Inference and Response Generation**

In [ ]:
from IPython.display import display, HTML

def cevap_uret(soru, instruction=None, max_new_tokens=300, max_input_tokens=1024,
              top_k=50, top_p=0.85, temperature=0.3, no_repeat_ngram_size=3):

    if instruction:
        user_content = f"{instruction}\n\n{soru}"
    else:
        user_content = soru

    messages = [{"role": "user", "content": user_content}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_input_tokens,
    ).to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,               # top_p / temperature'ın geçerli olması için
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        no_repeat_ngram_size=no_repeat_ngram_size,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "model" in text:
        return text.split("model", 1)[-1].strip()
    return text.strip()

def kutucuk_goster(soru, cevap, baslik="Yanıt"):
    soru_html = (
        str(soru)
        .replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        .replace("\n", "<br>")
    )
    cevap_html = (
        str(cevap)
        .replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        .replace("\n", "<br>")
    )

    html = f"""
    <div style="
        border:1px solid #3a3a3a;
        border-radius:12px;
        padding:14px 16px;
        margin:10px 0;
        background:#121212;
        color:#eaeaea;
        font-family: ui-sans-serif, system-ui, -apple-system, Segoe UI, Roboto, Arial;
        line-height:1.45;
        ">
      <div style="font-weight:700; font-size:14px; margin-bottom:8px;">{baslik}</div>
      <div style="opacity:0.9; font-size:13px; margin-bottom:10px;">
        <span style="font-weight:600;">Soru:</span><br>{soru_html}
      </div>
      <div style="font-size:13px;">
        <span style="font-weight:600;">Cevap:</span><br>{cevap_html}
      </div>
    </div>
    """
    display(HTML(html))

# **Q & A**

In [ ]:
instruction = "You are a medical expert. Provide professional guidance and recommend seeking a clinician for diagnosis."
soru = "I have asthma, hypertension, and diabetes. Can I take NSAIDs for joint pain, or should I consider alternative medications?"

cevap = cevap_uret(soru, instruction=instruction)
kutucuk_goster(soru, cevap, baslik="Medical QA Yanıtı")

In [ ]:
instruction = "You are a medical expert. Provide professional guidance and recommend seeking a clinician for diagnosis."
soru = (
    "I have a 5 month old baby who is very congested with a terrible cough. "
    "Its rattly/raspy and croupy sounding cough. She started choking on her coughs "
    "and the mucous that has come up. She also has a fever and runny nose. "
    "Should i take her to urgent care?"
)
cevap = cevap_uret(soru, instruction=instruction)
kutucuk_goster(soru, cevap, baslik="Medical QA Yanıtı")

In [ ]:
instruction = "You are a medical expert. Provide professional guidance and recommend seeking a clinician for diagnosis."
soru = (
    "I’ve been feeling anxious and tired since my thyroid levels went abnormal. "
    "Could my thyroid issues be causing these symptoms, and how should I approach treatment?"
)
cevap = cevap_uret(soru, instruction=instruction)
kutucuk_goster(soru, cevap, baslik="Medical QA Yanıtı")

# **Uploading LoRA Adapters to Hugging Face Hub (Share & Reuse)**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi

LOCAL_ADAPTER_DIR = "/content/drive/MyDrive/Colab Notebooks/MR/gemmaqa_project/Final Model"
HF_REPO_ID = "azizdeniz890/gemma-medical-qa-lora"  # bunu değiştir
api = HfApi()
api.create_repo(repo_id=HF_REPO_ID, exist_ok=True, private=False)

api.upload_folder(
    folder_path=LOCAL_ADAPTER_DIR,
    repo_id=HF_REPO_ID,
    repo_type="model"
)
print("Yüklendi:", HF_REPO_ID)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...nal Model/tokenizer.model: 100%|##########| 4.24MB / 4.24MB            

  ...adapter_model.safetensors:   1%|          |  556kB / 83.1MB            

Yüklendi: azizdeniz890/gemma-medical-qa-lora


# **Plug-and-Play Model Loading (No Training Required)**

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL = "google/gemma-2-2b-it"
ADAPTER_REPO = "azizdeniz890/gemma-medical-qa-lora"  # senin hub repo id

if not torch.cuda.is_available():
    raise RuntimeError("GPU önerilir. CUDA bulunamadı.")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, token=True)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    quantization_config=bnb_config,
    token=True
)

model = PeftModel.from_pretrained(base_model, ADAPTER_REPO)
model.eval()

print("Base model + LoRA adapter yüklendi.")

In [ ]:
def cevap_uret(soru, instruction=None, max_new_tokens=250, top_p=0.85, temperature=0.3):
    if instruction:
        user_content = f"{instruction}\n\n{soru}"
    else:
        user_content = soru

    messages = [{"role": "user", "content": user_content}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)

    out = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_p=top_p,
        temperature=temperature,
        no_repeat_ngram_size=3,
    )

    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text.split("model", 1)[-1].strip() if "model" in text else text.strip()

instruction = "You are a medical expert. Provide professional guidance and recommend seeking a clinician for diagnosis."
soru = "I have asthma, hypertension, and diabetes. Can I take NSAIDs for joint pain, or should I consider alternatives?"

print(cevap_uret(soru, instruction=instruction))